# Data Analysis for profitable free app on Android and iOS

This is a data analysis project for a free app development company. Our goal here is to analyse the various influencing factors that drive users to download free apps from *iOS App Store* and *Android Play Store*. We will analyze data for existing free applications to identify the type, genere and other categorization of apps that are most profitable.

This being a free app our only source of revenue is ads. Hence we will understand and map the features that drive users download a app and to stay loyal to the apps in spite of having to interact with ads. Also catalogue additional features that will sustain and grow this loyaltity and satisfaction, as such **retaining returning customers in addition to attracting new first time**.

## Data Gathering

To understand the features to implement in the new app. We will first analyze exiting app information from the Apple Store and Google Play store. However considering that there are millions of apps to be considere we will work on a small subset of those apps for the purpose of this project. Below is the links to the data sets used for this purpose:

1. A data set containing data of about approximately 10,000 Android apps from Google Play; the data was collected in August 2018. You can download the data set directly from [this link](https://dq-content.s3.amazonaws.com/350/googleplaystore.csv).


2. A data set containing data about approximately 7,000 iOS apps from the App Store; the data was collected in July 2017. You can download the data set directly from [this link](https://dq-content.s3.amazonaws.com/350/AppleStore.csv).

In [1]:
def explore_data(data_set, start = 0, end = 5, row_column_count = True):
    print(data_set[0], '\n')
    data_subset = data_set[start + 1:end]
    for row in data_subset:
        print(row, '\n')
    if row_column_count:
        print('# of columns:',len(data_set[0]))
        print('# of rows:',len(data_set))

In [2]:
open_file = open('googleplaystore.csv')
from csv import reader
read_file = reader(open_file)
android = list(read_file)
explore_data(android)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver'] 

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up'] 

['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up'] 

['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up'] 

['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up'] 

# of columns: 13
# of rows: 10842


In [3]:
open_file = open('AppleStore.csv')
from csv import reader
read_file = reader(open_file)
apple = list(read_file)
explore_data(apple)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic'] 

['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1'] 

['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1'] 

['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1'] 

['420009108', 'Temple Run', '65921024', 'USD', '0.0', '1724546', '3842', '4.5', '4.0', '1.6.2', '9+', 'Games', '40', '5', '1', '1'] 

# of columns: 16
# of rows: 7198


In [4]:
def duplicates(data_set, column):
    duplicate_set = []
    unique_set = []
    for row in data_set[1:]:
        name = row[column]
        if name in unique_set:
            duplicate_set.append(name)
        else:
            unique_set.append(name)
    print("Duplicate Apps: ", len(duplicate_set))
    print("Unique Apps: ", len(unique_set))

In [5]:
def incomplete_data(data_set):
    length = len(data_set[0])
    for row in data_set[1:]:
        if len(row) < length:
            data_set.remove(row)
            print('Deleted: ', row)

In [6]:
incomplete_data(android)

Deleted:  ['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


In [7]:
incomplete_data(apple)

In [8]:
duplicates(android, 0)

Duplicate Apps:  1181
Unique Apps:  9659


In [9]:
duplicates(apple, 1)

Duplicate Apps:  2
Unique Apps:  7195


In [10]:
def unique_dict(data_set, n, r, v):
    review_filter = {}
    for row in data_set[1:]:
        name = row[n]
        reviews = row[r]
        version = row[v]
        if name in review_filter:
            if version > review_filter[name][0]:
                review_filter[name][0] = version
                review_filter[name][1] = reviews
            elif version == review_filter[name][0] and reviews > review_filter[name][1]:
                review_filter[name][1] = reviews
        else:
            review_filter[name] = [version, reviews]
    return review_filter

In [11]:
def get_unique(data_set, unique_data_dict, n, r, v):
    clean_data = [data_set[0]]
    already_added = []
    for row in data_set[1:]:
        name = row[n]
        reviews = row[r]
        version = row[v]
        if name not in already_added:
            if unique_data_dict[name][0] == version and unique_data_dict[name][1] == reviews:
                clean_data.append(row)
                already_added.append(name)
    return clean_data

In [12]:
android_dict = unique_dict(android, 0, 3, 11)

dedupe_android_set = get_unique(android, android_dict, 0, 3, 11)

print(dedupe_android_set[:6])

print(len(dedupe_android_set))

[['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver'], ['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up'], ['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up'], ['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up'], ['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', '100,000+', 'Free', '0', 'Everyone', 'Art & Design;Creativity', 'June 20, 2018', '1.1', '4.4 and up'], ['Paper flowers instructions', 'ART_AND_DESIGN', '4.4', '167', '5.6M', '50,000+', 'Free', '0', 'Everyone'

In [13]:
apple_dict = unique_dict(apple, 1, 5, 9)

dedupe_apple_set = get_unique(apple, apple_dict, 1, 5, 9)

print(dedupe_apple_set[:6])

print(len(dedupe_apple_set))

[['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic'], ['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1'], ['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1'], ['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1'], ['420009108', 'Temple Run', '65921024', 'USD', '0.0', '1724546', '3842', '4.5', '4.0', '1.6.2', '9+', 'Games', '40', '5', '1', '1'], ['284035177', 'Pandora - Music & Radio', '130242560', 'USD', '0.0', '1126879', '3594', '4.0', '4.5', '8.4.1', '12+', 'Music', '37', '4', '1', '1']]
7196


In [14]:
def non_english(data_set, n):
    english_data = [data_set[0]]
    for row in data_set[1:]:
        name = row[n]
        count = 0
        for character in name:
            if ord(character) > 127:
                count += 1
        if count <= 3: 
            english_data.append(row)
    return english_data

In [15]:
english_android = non_english(dedupe_android_set, 0)
print(english_android[:6])
print(len(english_android))

[['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver'], ['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up'], ['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up'], ['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up'], ['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', '100,000+', 'Free', '0', 'Everyone', 'Art & Design;Creativity', 'June 20, 2018', '1.1', '4.4 and up'], ['Paper flowers instructions', 'ART_AND_DESIGN', '4.4', '167', '5.6M', '50,000+', 'Free', '0', 'Everyone'

In [16]:
english_apple = non_english(dedupe_apple_set, 1)
print(english_apple[:6])
print(len(english_apple))

[['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic'], ['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1'], ['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1'], ['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1'], ['420009108', 'Temple Run', '65921024', 'USD', '0.0', '1724546', '3842', '4.5', '4.0', '1.6.2', '9+', 'Games', '40', '5', '1', '1'], ['284035177', 'Pandora - Music & Radio', '130242560', 'USD', '0.0', '1126879', '3594', '4.0', '4.5', '8.4.1', '12+', 'Music', '37', '4', '1', '1']]
6182


In [17]:
def free_app(data_set, f):
    free_set = [data_set[0]]
    for row in data_set[1:]:
        cost = row[f]
        if cost == '0' or cost == '0.0':
            free_set.append(row)
    return free_set

In [18]:
final_android = free_app(english_android, 7)
print(final_android[:6])
print(len(final_android))

[['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver'], ['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up'], ['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up'], ['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up'], ['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', '100,000+', 'Free', '0', 'Everyone', 'Art & Design;Creativity', 'June 20, 2018', '1.1', '4.4 and up'], ['Paper flowers instructions', 'ART_AND_DESIGN', '4.4', '167', '5.6M', '50,000+', 'Free', '0', 'Everyone'

In [19]:
final_apple = free_app(english_apple, 4)
print(final_apple[:6])
print(len(final_apple))

[['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic'], ['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1'], ['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1'], ['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1'], ['420009108', 'Temple Run', '65921024', 'USD', '0.0', '1724546', '3842', '4.5', '4.0', '1.6.2', '9+', 'Games', '40', '5', '1', '1'], ['284035177', 'Pandora - Music & Radio', '130242560', 'USD', '0.0', '1126879', '3594', '4.0', '4.5', '8.4.1', '12+', 'Music', '37', '4', '1', '1']]
3221


In [50]:
def get_freq(data_set, i):
    value_freq = {}
    for row in data_set[1:]:
        value = row[i]
        if value in value_freq:
            value_freq[value] += 1
        else:
            value_freq[value] = 1
    
    total = len(data_set)
    percent_table = {}
    for key, value in value_freq.items():
        percent_table[key] = round(((value / total) * 100), 2)
    
    sort_percentage = {k: v for k, v in sorted(percent_table.items(), key = lambda item: item[1], reverse = True)}
    for key, value in sort_percentage.items():
        print(key, " : ", value)

In [54]:
android_genre = get_freq(final_android, 1)

FAMILY  :  18.9
GAME  :  9.72
TOOLS  :  8.46
BUSINESS  :  4.58
LIFESTYLE  :  3.9
PRODUCTIVITY  :  3.89
FINANCE  :  3.7
MEDICAL  :  3.52
SPORTS  :  3.4
PERSONALIZATION  :  3.32
COMMUNICATION  :  3.25
HEALTH_AND_FITNESS  :  3.08
PHOTOGRAPHY  :  2.94
NEWS_AND_MAGAZINES  :  2.8
SOCIAL  :  2.66
TRAVEL_AND_LOCAL  :  2.34
SHOPPING  :  2.25
BOOKS_AND_REFERENCE  :  2.14
DATING  :  1.86
VIDEO_PLAYERS  :  1.79
MAPS_AND_NAVIGATION  :  1.4
FOOD_AND_DRINK  :  1.24
EDUCATION  :  1.17
ENTERTAINMENT  :  0.96
LIBRARIES_AND_DEMO  :  0.94
AUTO_AND_VEHICLES  :  0.93
HOUSE_AND_HOME  :  0.82
WEATHER  :  0.8
EVENTS  :  0.71
PARENTING  :  0.65
ART_AND_DESIGN  :  0.64
COMICS  :  0.62
BEAUTY  :  0.6


In [56]:
android_genre = get_freq(final_android, -4)

Tools  :  8.45
Entertainment  :  6.07
Education  :  5.35
Business  :  4.58
Lifestyle  :  3.89
Productivity  :  3.89
Finance  :  3.7
Medical  :  3.52
Sports  :  3.46
Personalization  :  3.32
Communication  :  3.25
Action  :  3.1
Health & Fitness  :  3.08
Photography  :  2.94
News & Magazines  :  2.8
Social  :  2.66
Travel & Local  :  2.32
Shopping  :  2.25
Books & Reference  :  2.14
Simulation  :  2.04
Dating  :  1.86
Arcade  :  1.86
Video Players & Editors  :  1.77
Casual  :  1.75
Maps & Navigation  :  1.4
Food & Drink  :  1.24
Puzzle  :  1.13
Racing  :  0.99
Libraries & Demo  :  0.94
Role Playing  :  0.94
Auto & Vehicles  :  0.93
Strategy  :  0.91
House & Home  :  0.82
Weather  :  0.8
Events  :  0.71
Adventure  :  0.68
Comics  :  0.61
Art & Design  :  0.6
Beauty  :  0.6
Parenting  :  0.5
Card  :  0.45
Casino  :  0.43
Trivia  :  0.42
Educational;Education  :  0.39
Board  :  0.38
Educational  :  0.37
Education;Education  :  0.34
Word  :  0.26
Casual;Pretend Play  :  0.24
Music  :  0.2
E

In [57]:
apple_genre = get_freq(final_apple, 11)

Games  :  58.12
Entertainment  :  7.89
Photo & Video  :  4.97
Education  :  3.66
Social Networking  :  3.29
Shopping  :  2.61
Utilities  :  2.51
Sports  :  2.14
Music  :  2.05
Health & Fitness  :  2.02
Productivity  :  1.74
Lifestyle  :  1.58
News  :  1.33
Travel  :  1.24
Finance  :  1.12
Weather  :  0.87
Food & Drink  :  0.81
Reference  :  0.56
Business  :  0.53
Book  :  0.43
Navigation  :  0.19
Medical  :  0.19
Catalogs  :  0.12


In [60]:
def avg_user(data_set, genre_dict, g, u): 
    for genre in genre_dict:
        total = 0
        user_ratings = 0
        for row in data_set:
            data_genre = row[g]
            if data_genre == genre:
                user_ratings += row[u]
                total += 1
    print(genre, ":", (user_ratings / total))

In [ ]:
avg_user(, android_genre, 1, )